In [0]:
dbutils.widgets.text("catalog", "", "The catalog to use")
dbutils.widgets.text("schema", "", "The schema to use")
dbutils.widgets.text("warehouse_id", "", "Warehouse for reporting")
dbutils.widgets.text("prefix", "", "Prefix for all asset display names")

Conditionally Create Catalog and Schema

In [0]:
catalog = dbutils.widgets.get("catalog")
schema = dbutils.widgets.get("schema")

# Check and create catalog if not exists
spark.sql(f"CREATE CATALOG IF NOT EXISTS {catalog}")

# Check and create schema if not exists
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {catalog}.{schema}")

Update DAB to use Variables

In [0]:
catalog = dbutils.widgets.get("catalog")
schema = dbutils.widgets.get("schema")
warehouse_id = dbutils.widgets.get("warehouse_id")
prefix = dbutils.widgets.get("prefix")

# Build variable arguments for DAB deployment
var_args = f"--var='catalog={catalog}' --var='schema={schema}' --var='warehouse_id={warehouse_id}' --var='prefix={prefix}'"

print("DAB Variable Configuration:")
print(f"  catalog: {catalog}")
print(f"  schema: {schema}")
print(f"  warehouse_id: {warehouse_id}")
print(f"  prefix: {prefix}")
print(f"\nVariable args for CLI: {var_args}")


Deploy DAB

In [0]:
import subprocess
import os

# Get the bundle root path (assuming notebook is in src/)
bundle_root = os.path.dirname(os.getcwd())

# Deploy the DAB bundle with variables
deploy_cmd = f"""
cd {bundle_root} && \
databricks bundle deploy --target demo \
  --var='catalog={catalog}' \
  --var='schema={schema}' \
  --var='warehouse_id={warehouse_id}' \
  --var='prefix={prefix}'
"""

print(f"Deploying DAB from: {bundle_root}")
print(f"Running: {deploy_cmd}")

result = subprocess.run(deploy_cmd, shell=True, capture_output=True, text=True)

print("\n--- STDOUT ---")
print(result.stdout)

if result.returncode != 0:
    print("\n--- STDERR ---")
    print(result.stderr)
    raise Exception(f"DAB deployment failed with return code {result.returncode}")
else:
    print("\n✅ DAB deployment completed successfully!")


Load Initial Data

In [0]:
# Load initial sample data by running the Games Social Listening Demo Job via DAB
# This will pull data for the default game (Pokemon Go from Google Play)

print("Loading initial data...")
print(f"Catalog: {catalog}")
print(f"Schema: {schema}")

# Run the job using databricks bundle run
run_cmd = f"""
cd {bundle_root} && \
databricks bundle run Games_Social_Listening_Demo_Job \
  --var='catalog={catalog}' \
  --var='schema={schema}' \
  --var='warehouse_id={warehouse_id}' \
  --var='prefix={prefix}'
"""

print(f"Running: {run_cmd}")

result = subprocess.run(run_cmd, shell=True, capture_output=True, text=True)

print("\n--- STDOUT ---")
print(result.stdout)

if result.returncode != 0:
    print("\n--- STDERR ---")
    print(result.stderr)
    raise Exception(f"Job run failed with return code {result.returncode}")

print(f"\n✅ Initial data load completed!")

# Display setup complete summary
print("""
================================================================================
🎉 Setup Complete!
================================================================================

Your Games Social Listening pipeline is now deployed. Here's what was configured:

  ✓ Catalog & Schema: Created in Unity Catalog
  ✓ DAB Resources: Pipeline, Job, and Dashboard deployed to your workspace
  ✓ Initial Data: Sample Google Play reviews loaded for Pokemon Go

Next Steps:
  1. Navigate to the deployed Dashboard to view initial insights
  2. Add more games by running the 'Games Social Listening Demo Job' with different parameters
  3. Schedule the job to automatically refresh data on your preferred cadence
================================================================================
""")
